In [14]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import DeepLake
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb
from chromadb.config import Settings
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub

from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import SystemMessage, HumanMessage, AIMessage

from langchain.llms import GPT4All
# from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from tqdm import tqdm
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator, field_validator
from pydantic import BaseModel, Field
from typing import List
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import cohere
from langchain.embeddings import CohereEmbeddings
from langchain_community.llms import Cohere
from langchain_community.chat_models import ChatCohere
from langchain.chains import SimpleSequentialChain
from langchain.output_parsers import CommaSeparatedListOutputParser


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [4]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ["ACTIVELOOP_TOKEN"] = ""
os.environ["GOOGLE_API_KEY"]= ""
os.environ["GOOGLE_CSE_ID"]= ""
os.environ["HUGGINGFACEHUB_API_TOKEN"]= ""
os.environ["COHERE_API_KEY"] = ""
os.environ["WOLFRAM_ALPHA_APPID"] = ""
os.environ["SERPAPI_API_KEY"]= ""

# What is Langchain?

![title](../data/llm_os_1.png)

![title](../data/llm_os_1.png)

![title](../data/llm_os_2.png)

__LangChain__ is an innovative framework which interacts with all the modules of LLM OS and tailored for developing applications that leverage the capabilities of language models. \
 it's a full-fledged ecosystem comprising several integral parts. 
 * __LangChain__: helps in development of the application
 * __LangSmith__: helps in inspecting, testing, and monitoring the chains, ensuring that the applications are constantly improving and ready for deployment.
 * __LangServe__: helps in deploying or serving the application

LangChain allows for the creation of language model applications through various modules. These modules are:

* __Model I/O__: Facilitates interaction with various language models, handling their inputs and outputs efficiently.
* __Retrieval__: Enables access to and interaction with application-specific data, crucial for dynamic data utilization.
* __Chains__: Offers pre-defined, reusable compositions that serve as building blocks for application development.
* __Memory__: Maintains application state across multiple chain executions, essential for context-aware interactions.
* __Tools__: Tools help users to effortlessly tap into a diverse range of functionalities and information sources to tackle challenges and generate meaningful responses.
* __Agents__: Empower applications to select appropriate tools based on high-level directives, enhancing decision-making capabilities.

# 1. Model I/O

In LangChain, the core element of any application revolves around the language model. This module provides the essential building blocks to interface effectively with any language model, ensuring seamless integration and communication.\
__Key Components of Model I/O__
1. __LLMs and Chat Models (used interchangeably)__:
    * __LLMs__:
        * Definition: Pure text completion models.
        * Input/Output: Take a text string as input and return a text string as output.
    * __Chat Models__
        * Definition: Models that use a language model as a base but differ in input and output formats.
        * Input/Output: Accept a list of chat messages as input and return a Chat Message.
2. __Prompts__: Templatize, dynamically select, and manage model inputs. Allows for the creation of flexible and context-specific prompts that guide the language model's responses.
3. __Output Parsers__: Extract and format information from model outputs. Useful for converting the raw output of language models into structured data or specific formats needed by the application.




## LLMs

In [5]:
llm_openai = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.0)
chat_openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

LangChain offers a uniform interface to interact with various LLMs like OpenAI, Cohere, Hugging Face, etc.



In [22]:
from langchain.llms import OpenAI
# llm_openai = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.0)
llm_hf = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':0.0})
# llm = Cohere(model="command")
llm_cohere = Cohere()

In [23]:
query = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print("==================OPENAI GPT3.5 model==========================")
print(llm_openai(query))
print("==================HUGGIGFACE flan-t5-large model==========================")
print(llm_hf(query))
print("==================COHERE DEFAULT model==========================")
print(llm_cohere(query))

==================OPENAI GPT3.5 model==========================


Monday:
Warm-up: 10 minutes of brisk walking or jogging
Main workout:
- 30 minutes of cycling on a hilly route
- 3 sets of 10 push-ups
- 3 sets of 10 squats
- 3 sets of 10 lunges
Cool down: 5 minutes of stretching

Tuesday:
Warm-up: 10 minutes of jumping jacks or jump rope
Main workout:
- 30 minutes of running on a trail or track
- 3 sets of 10 burpees
- 3 sets of 10 mountain climbers
- 3 sets of 10 tricep dips using a park bench
Cool down: 5 minutes of stretching

Wednesday:
Rest day or low-intensity activity such as yoga or walking

Thursday:
Warm-up: 10 minutes of high knees or butt kicks
Main workout:
- 30 minutes of swimming laps in a pool or open water
- 3 sets of 10 pull-ups using a playground bar
- 3 sets of 10 jump squats
- 3 sets of 10 Russian twists
Cool down: 5 minutes of stretching

Friday:
Warm-up: 10 minutes of jogging in place or jumping jacks
Main workout:

==================HUGGIGFACE flan-t5-large mode

In [10]:
query = "What is a good name for a company that makes eco-friendly water bottles."
print("==================OPENAI GPT3.5 model==========================")
print(llm_openai(query))
print("==================HUGGIGFACE flan-t5-large model==========================")
print(llm_hf(query))
print("==================COHERE DEFAULT model==========================")
print(llm_cohere(query))

==================OPENAI GPT3.5 model==========================


EcoHydrate Co.
==================HUGGIGFACE flan-t5-large model==========================
reusable
==================COHERE DEFAULT model==========================
 Here are a few ideas for names for a company that makes eco-friendly water bottles:

1. Green Guardian: This name suggests that the company is protecting the environment by providing eco-friendly products.

2. Aqua Verde: This name combines the Spanish words for "water" and "green," conveying a clear message about the company's focus on environmentally friendly water bottles.

3. EcoHydrate: A catchy name that combines "eco" for ecology and "hydrate" for the purpose of the product, this name is easy to remember and conveys the dual purpose of environmental friendliness and hydration.

4. EarthtoPouch: This name is a play on the word "earth" and "pouch," suggesting an earth-friendly water bottle product. 

5. Blue Planet: This name is a direct reference to the

In [12]:
# query = "List of seven wonders of world"
# print("==================OPENAI GPT3.5 model==========================")
# print(llm_openai(query))
# print("==================HUGGIGFACE flan-t5-large model==========================")
# print(llm_hf(query))
# print("==================COHERE DEFAULT model==========================")
# print(llm_cohere(query))

## Chat Models

LangChain's integration with chat models, a specialized variation of language models, is essential for creating interactive chat applications. While they utilize language models internally, chat models present a distinct interface centered around chat messages as inputs and outputs. 

In [16]:
# chat_openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
# cohere_chat_model = ChatCohere()
messages = [
    SystemMessage(content="You are Micheal Jordan."),
    HumanMessage(content="Which shoe manufacturer are you associated with?"),
]
response = chat_openai(messages)
print(response.content)

I am associated with Nike.


In [20]:
messages = [
    SystemMessage(content="You are an assistant that helps users find information about movies."),
    HumanMessage(content="Find information about the movie Schindler's List"),
]
response = chat_openai(messages)
print(response)
print(response.content)

"Schindler's List" is a 1993 American historical drama film directed by Steven Spielberg. The film is based on the novel "Schindler's Ark" by Thomas Keneally and tells the true story of Oskar Schindler, a German businessman who saved the lives of more than a thousand Polish-Jewish refugees during the Holocaust by employing them in his factories.

The film stars Liam Neeson as Oskar Schindler, Ben Kingsley as Itzhak Stern, and Ralph Fiennes as Amon Goeth. "Schindler's List" received critical acclaim and won seven Academy Awards, including Best Picture, Best Director, and Best Original Score.

The movie is known for its powerful portrayal of the horrors of the Holocaust and the heroism of those who risked their lives to save others. It is considered one of the greatest films ever made and is often included in lists of the best movies of all time.


In [22]:
messages.append(response)
prompt= HumanMessage(content= "Can you also recommend movies similar to this?")
messages.append(prompt)
response= chat_openai(messages)
print(response.content)

If you enjoyed "Schindler's List" and are looking for similar movies that explore themes of the Holocaust, World War II, and human resilience, here are some recommendations:

1. "The Pianist" (2002) - Directed by Roman Polanski, this film tells the true story of Wladyslaw Szpilman, a Polish-Jewish pianist who survived the Holocaust in Warsaw.

2. "Life is Beautiful" (1997) - An Italian film directed by and starring Roberto Benigni, which tells the story of a Jewish father who uses humor and imagination to protect his son in a concentration camp.

3. "The Boy in the Striped Pyjamas" (2008) - Based on the novel by John Boyne, this film follows the unlikely friendship between a young German boy and a Jewish boy in a concentration camp.

4. "Sophie's Choice" (1982) - Starring Meryl Streep and Kevin Kline, this film explores the devastating choices faced by a Polish woman during the Holocaust.

5. "The Diary of Anne Frank" (1959) - Based on the real-life diary of Anne Frank, this film depic

## Prompts

In [27]:
# Simple prompt with placeholders
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# Filling placeholders to create a prompt
filled_prompt = prompt_template.format(adjective="funny", content="robots")
print(filled_prompt)
print(llm_openai(filled_prompt))

Tell me a funny joke about robots.


Why did the robot go on a diet?

Because he wanted to reduce his "byte" size!


In [28]:
template = """Question: {question}

Answer: """

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"
filled_prompt = prompt.format(question=question)
print(llm_openai(filled_prompt))

The capital city of France is Paris.


In [35]:
# Defining a chat prompt with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# Formatting the chat prompt
formatted_messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(formatted_messages)
chat_openai(formatted_messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


AIMessage(content='My name is Bob. How can I assist you today?')

In [9]:
template = "You are an assistant that helps users find information about places."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about {place}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
formatted_messages= chat_prompt.format_messages(place="Ladakh")
response = chat_openai(formatted_messages)

print(response.content)

Ladakh is a region in the northern part of India, located in the state of Jammu and Kashmir. It is known for its stunning landscapes, high mountain ranges, and unique culture. Here are some key points about Ladakh:

1. Geography: Ladakh is situated between the Kunlun mountain range in the north and the main Great Himalayas to the south. It is a high-altitude desert with barren mountains, deep valleys, and crystal-clear lakes.

2. Culture: Ladakh has a rich cultural heritage influenced by Tibetan Buddhism. The region is dotted with monasteries, stupas, and prayer flags. The people of Ladakh are known for their warmth and hospitality.

3. Tourism: Ladakh is a popular tourist destination, especially for adventure enthusiasts. Visitors can enjoy activities such as trekking, mountaineering, river rafting, and camping. The famous Pangong Lake and Nubra Valley are must-visit attractions in Ladakh.

4. Climate: Ladakh experiences extreme weather conditions with cold winters and mild summers. D

## Output Parsers

While the language models can only generate textual outputs, a predictable data structure is always preferred in a production environment. 

The Output Parsers help create a data structure to define the expectations from the output precisely. We can ask for a list of words in case of a Thesaurus application or a combination of different variables like a word and the explanation of why it fits. The parser can extract the expected information for you.


### PydanticOutputParser

We always import and follow the necessary libraries by creating the Suggestions schema class. There are two essential parts to this class:
1. __Expected Outputs__: Each output is defined by declaring a variable with desired type, like a list of strings (: List[str]) in the sample code, or it could be a single string (: str) if you are expecting just one word/sentence as the response. Also, It is required to write a simple explanation using the Field function’s description attribute to help the model during inference. (We will see an example of having multiple outputs later in the lesson)
2. __field_validator__: It is possible to declare functions to validate the formatting. We ensure that the first character is not a number in the sample code. The function’s name is unimportant, but the @field_validator decorator must receive the same name as the variable you want to approve. (like @validator(’words’)) It is worth noting that the field variable inside the validator function will be a list if you specify it as one.

In [28]:
# Define your desired data structure using Pydantic
class Suggestions(BaseModel):
    place_name: List[str] = Field(description="list of places to travel based on preference")
    reasons: List[str] = Field(description="the reasoning for why this place fits the preference")
    
    @field_validator('place_name')
    def not_start_with_number(cls, v):
        for item in v:
            if item[0].isnumeric():
                raise ValueError("The name of the place can not start with numbers!")
        return v
    
    @field_validator('reasons')
    def not_more_than_3_sentences(cls, v):
        for idx, item in enumerate( v ):
            sentences= item.split(".")
            if len(sentences) >3:
                v[idx]= sentences[:3]
        return v
    

# Set up a PydanticOutputParser
pydantic_parser = PydanticOutputParser(pydantic_object=Suggestions)

template = """
You are an assistant that helps users to plan trips.
Suggest a list of places to visit in {country} for someone who loves {preference} with proper reasoning .
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["country", "preference"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

model_input = prompt.format(country="India", preference="mountains")
output = llm_openai(model_input)
pydantic_parser.parse(output)



Suggestions(place_name=['Leh', 'Manali', 'Darjeeling', 'Shimla', 'Gulmarg'], reasons=['Leh is known for its breathtaking views of the Himalayas and offers a unique cultural experience with its Tibetan Buddhist influence.', 'Manali is a popular destination for adventure seekers with its many trekking and skiing opportunities.', 'Darjeeling is a charming hill station with stunning views of the Kanchenjunga mountain range and is famous for its tea plantations.', 'Shimla is a picturesque hill station with colonial architecture and offers a peaceful retreat in the mountains.', 'Gulmarg is a paradise for skiing enthusiasts and offers stunning views of the snow-capped peaks of the Pir Panjal range.'])

In [31]:
# print(model_input)


You are an assistant that helps users to plan trips.
Suggest a list of places to visit in India for someone who loves mountains with proper reasoning .
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"place_name": {"description": "list of places to travel based on preference", "items": {"type": "string"}, "title": "Place Name", "type": "array"}, "reasons": {"description": "the reasoning for why this place fits the preference", "items": {"type": "string"}, "title": "Reasons", "type": "array"}}, "required": ["place_name", "reasons"]}
```



### JsonOutputParser

In [68]:
from langchain_core.output_parsers import JsonOutputParser

# Initialize the JSON parser
json_parser = JsonOutputParser()

template = """
You are an assistant that helps users to plan trips.
Suggest a list of places to visit in {country} for someone who loves {preference} with proper reasoning.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["country", "preference"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

model_input = prompt.format(country="India", preference="mountains")
output = llm_openai(model_input)
json_parser.parse(output)



{'places': [{'name': 'Leh-Ladakh',
   'reason': 'Known for its breathtaking landscapes, high altitude passes, and adventure activities like trekking and mountaineering.'},
  {'name': 'Manali',
   'reason': 'A popular hill station with stunning views of the Himalayas, perfect for skiing, paragliding, and other outdoor activities.'},
  {'name': 'Darjeeling',
   'reason': 'Famous for its tea plantations, scenic views of the Kanchenjunga mountain, and the Darjeeling Himalayan Railway.'},
  {'name': 'Shimla',
   'reason': 'A charming hill station with colonial architecture, surrounded by snow-capped mountains and offering activities like skiing and trekking.'},
  {'name': 'McLeod Ganj',
   'reason': 'Located in the foothills of the Dhauladhar range, this town is known for its Tibetan culture, monasteries, and trekking opportunities.'}]}

### CommaSeparatedListOutputParser

In [72]:

# Initialize the CSV parser
csv_parser = CommaSeparatedListOutputParser()

template = """
You are an assistant that helps users to plan trips.
Suggest a list of places to visit in {country} for someone who loves {preference}.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["country", "preference"],
    partial_variables={"format_instructions": csv_parser.get_format_instructions()}
)

model_input = prompt.format(country="India", preference="mountains")
output = llm_openai(model_input)
csv_parser.parse(output)



['Manali',
 'Shimla',
 'Leh',
 'Darjeeling',
 'Sikkim',
 'McLeod Ganj',
 'Auli',
 'Nainital',
 'Mussoorie',
 'Gangtok']

# 2. Retrieval (RAG)


Retrieval in LangChain plays a crucial role in applications that require user-specific data, not included in the model's training set. This process, known as Retrieval Augmented Generation (RAG), involves fetching external data and integrating it into the language model's generation process. 

LangChain provides a comprehensive suite of tools and functionalities to facilitate this process, catering to both simple and complex applications.



## Document Loaders


Document loaders in LangChain enable the extraction of data from various sources. All these loaders ingest data into __Document__ classes.

In [6]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyPDFLoader


path= "/home/jupyter/self_learning/Langchain/data/"
############ Text File Loader #############
text_loader = TextLoader(path+"my_file.txt")
text_doc = text_loader.load()

############ CSV File Loader #############
csv_loader = CSVLoader(path+'mo_val_cg_100k_mexico_items_10k_sample_3k.csv')
csv_doc = csv_loader.load()

############ PDF File Loader #############
pdf_loader = PyPDFLoader(path+"Resume_Naquib_Alam.pdf")
pdf_doc = pdf_loader.load()



In [116]:
# csv_doc[0].page_content.split("\n")

Apart from these standard __Document Loaders__ LangChain offers a wide variety of custom loaders to directly load data from your apps (such as __Slack, Figma, Notion, Confluence, Google Drive, HTML dcouments__, etc.) and databases and use them in LLM applications.

## Document Transformers

Document transformers in LangChain are essential tools designed to manipulate documents, which we created in our previous subsection.

They are used for tasks such as splitting long documents into smaller chunks, combining, and filtering, which are crucial for adapting documents to a model's context window or meeting specific application needs.

### Character Text Splitter

In [7]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20, length_function=len)
cts_docs = text_splitter.split_documents(text_doc)

print (f"You have {len(cts_docs)} documents")
print (cts_docs)


Created a chunk of size 373, which is longer than the specified 100


You have 2 documents
[Document(page_content='Google opens up its AI language model PaLM to challenge OpenAI and GPT-3\nGoogle is offering developers access to one of its most advanced AI language models: PaLM.\nThe search giant is launching an API for PaLM alongside a number of AI enterprise tools\nit says will help businesses “generate text, images, code, videos, audio, and more from\nsimple natural language prompts.”', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='PaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or\nMeta’s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs,\nPaLM is a flexible system that can potentially carry out all sorts of text generation and\nediting tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for\nexample, or you could use it for tasks like summarizing text or even writing code.\n(It’s similar to features Googl

### Recursive Character Text Splitter

The Recursive Character Text Splitter is a text splitter designed to split the text into chunks based on a list of characters provided. It attempts to split text using the characters from a list in order until the resulting chunks are small enough. By default, the list of characters used for splitting is __["\n\n", "\n", " ", "]__, which tries to keep paragraphs, sentences, and words together as long as possible, as they are generally the most semantically related pieces of text. This means that the class first tries to split the text into two new-line characters. If the resulting chunks are still larger than the desired chunk size, it will then try to split the output by a single new-line character, followed by a space character, and so on, until the desired chunk size is achieved.

To use the RecursiveCharacterTextSplitter, you can create an instance of it and provide the following parameters:
* __chunk_size__ : The maximum size of the chunks, as measured by the length_function (default is 100).
* __chunk_overlap__: The maximum overlap between chunks to maintain continuity between them (default is 20).
* __length_function__: parameter is used to calculate the length of the chunks. By default, it is set to len, which counts the number of characters in a chunk. However, you can also pass a token counter or any other function that calculates the length of a chunk based on your specific requirements.

Using a token counter instead of the default __len__ function can benefit specific scenarios, such as when working with language models with token limits. For example, OpenAI's GPT-3 has a token limit of 4096 tokens per request, so you might want to count tokens instead of characters to better manage and optimize your requests.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, length_function=len)
rcts_docs = text_splitter.split_documents(text_doc)
print (f"You have {len(rcts_docs)} documents")
print (rcts_docs)


You have 12 documents
[Document(page_content='Google opens up its AI language model PaLM to challenge OpenAI and GPT-3', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='Google is offering developers access to one of its most advanced AI language models: PaLM.', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='The search giant is launching an API for PaLM alongside a number of AI enterprise tools', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='it says will help businesses “generate text, images, code, videos, audio, and more from', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='simple natural language prompts.”', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content='PaLM is a large language model, or LLM, similar to the 

### TokenTextSplitter

The main advantage of using TokenTextSplitter over other text splitters, like CharacterTextSplitter, is that it respects the token boundaries, ensuring that the chunks do not split tokens in the middle. This can be particularly helpful in maintaining the semantic integrity of the text when working with language models and embeddings.

This type of splitter breaks down raw text strings into smaller pieces by initially converting the text into BPE (Byte Pair Encoding) tokens, and subsequently dividing these tokens into chunks. It then reassembles the tokens within each chunk back into text.

In [16]:
from langchain.text_splitter import TokenTextSplitter

# Initialize the TokenTextSplitter with desired chunk size and overlap
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=20)

# Split into smaller chunks
tts_docs = text_splitter.split_documents(text_doc)
print (f"You have {len(tts_docs)} documents")
print (tts_docs)


You have 3 documents
[Document(page_content='Google opens up its AI language model PaLM to challenge OpenAI and GPT-3\nGoogle is offering developers access to one of its most advanced AI language models: PaLM.\nThe search giant is launching an API for PaLM alongside a number of AI enterprise tools\nit says will help businesses “generate text, images, code, videos, audio, and more from\nsimple natural language prompts.”\n\nPaLM is a large language model, or LLM,', metadata={'source': '/home/jupyter/self_learning/Langchain/data/my_file.txt'}), Document(page_content=' natural language prompts.”\n\nPaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or\nMeta’s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs,\nPaLM is a flexible system that can potentially carry out all sorts of text generation and\nediting tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for\nexample', metadata={'source': '/home/jupy

The __chunk_size__ parameter sets the maximum number of BPE tokens in each chunk, while __chunk_overlap__ defines the number of overlapping tokens between adjacent chunks. By modifying these parameters, you can fine-tune the granularity of the text chunks.

One potential drawback of using ___TokenTextSplitter___ is that it may require additional computation when converting text to BPE tokens and back. If you need a faster and simpler text-splitting method, you might consider using CharacterTextSplitter, which directly splits the text based on character count, offering a more straightforward approach to text segmentation.

## Text Embedding Models

Text embedding models in LangChain provide a standardized interface for various embedding model providers like __OpenAI, Cohere, Hugging Face, etc.__ These models transform text into vector representations, enabling operations like semantic search through text similarity in vector space.



In [9]:
def get_topk_similar_docs(model, query, docs, topk=5):
    # Generate embeddings for the documents and query
    document_embeddings = model.embed_documents(docs)
    query_embedding = model.embed_query(query)
    # Calculate similarity scores
    similarity_scores = cosine_similarity([query_embedding], document_embeddings)[0]
    # Find top-k similar document
    sorted_similarity_scores_idx = np.argsort(-similarity_scores)[:topk]
    topk_docs= [docs[i] for i in sorted_similarity_scores_idx]
    return topk_docs

### OpenAI Embedding

In [133]:
## OpenAI Embedding
# Define the documents
print("=========OpenAI Embeddings===========")
documents = [
    "The cat is on the mat.",
    "There is a cat on the mat.",
    "The dog is in the yard.",
    "There is a dog in the yard.",
]

query = "A cat is sitting on a mat."
k= 2
# Initialize the OpenAIEmbeddings instance
openai_embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

topk_similar_docs = get_topk_similar_docs(openai_embed_model, query, documents, k)

print(f"top-{k} similar document to the query '{query}':")
print(topk_similar_docs)


=========OpenAI Embeddings===========
Most similar document to the query 'A cat is sitting on a mat.':
['The cat is on the mat.', 'There is a cat on the mat.']


### HuggingFace Embedding

In [137]:
## HuggingFace Embedding
print("=========HuggingFace Embeddings===========")

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda:0'}
mpnet_embed_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

topk_similar_docs = get_topk_similar_docs(mpnet_embed_model, query, documents, k)

print(f"top-{k} similar document to the query '{query}':")
print(topk_similar_docs)


=========HuggingFace Embeddings===========
top-2 similar document to the query 'A cat is sitting on a mat.':
['The cat is on the mat.', 'There is a cat on the mat.']


### Cohere Embedding

In [142]:
## Cohere Embedding
print("=========Cohere Embeddings===========")

# Define a list of texts
documents= [
    "Hello from Cohere!", 
    "مرحبًا من كوهير!", 
    "Hallo von Cohere!",  
    "Bonjour de Cohere!", 
    "¡Hola desde Cohere!", 
    "Olá do Cohere!",  
    "Ciao da Cohere!", 
    "您好，来自 Cohere！", 
    "कोहेरे से नमस्ते!",
    "Paris is a very beautiful city.",
    "París es una ciudad muy hermosa.",
    "Paris est une très belle ville.",
    "पेरिस बहुत खूबसूरत शहर है."
]

query_lst = ["Hello from Cohere!", "पेरिस बहुत खूबसूरत शहर है.",  "您好，来自 Cohere！"]
k=5 
# Initialize the CohereEmbeddings object
cohere_ml_embed_model = CohereEmbeddings(model="embed-multilingual-v2.0")

topk_similar_docs = get_topk_similar_docs(cohere_ml_embed_model, query_lst[1], documents, k)

print(f"top-{k} similar document to the query '{query_lst[1]}':")
print(topk_similar_docs)


=========Cohere Embeddings===========
top-5 similar document to the query 'पेरिस बहुत खूबसूरत शहर है.':
['पेरिस बहुत खूबसूरत शहर है.', 'Paris is a very beautiful city.', 'Paris est une très belle ville.', 'París es una ciudad muy hermosa.', 'कोहेरे से नमस्ते!']


## Vector Stores

Vector stores in LangChain support the efficient storage and searching of text embeddings. LangChain integrates with over 50 vector stores, providing a standardized interface for ease of use.



In [168]:
chromadb_client = chromadb.Client()
db_cohere_ml_embed = Chroma.from_texts(
    documents, cohere_ml_embed_model, client=chromadb_client, collection_name="tmp_collection"
)

print("There are", db_cohere_ml_embed._collection.count(), "in the collection")

topk_similar_docs = db_cohere_ml_embed.similarity_search(query_lst[1], k=5) ##By default chromadb returns 4 documents

print(f"top-{k} similar document to the query '{query_lst[1]}':")
print(topk_similar_docs)

# chromadb_client.delete_collection(name="tmp_collection")

There are 13 in the collection
top-5 similar document to the query 'पेरिस बहुत खूबसूरत शहर है.':
[Document(page_content='पेरिस बहुत खूबसूरत शहर है.'), Document(page_content='Paris is a very beautiful city.'), Document(page_content='Paris est une très belle ville.'), Document(page_content='París es una ciudad muy hermosa.'), Document(page_content='कोहेरे से नमस्ते!')]


## Retrievers

Retrievers in LangChain are interfaces that return documents in response to an unstructured query. They are more general than vector stores, focusing on retrieval rather than storage. Although vector stores can be used as a retriever's backbone, there are other types of retrievers as well.

In [169]:
retriever = db_cohere_ml_embed.as_retriever(search_kwargs={"k": 5})
topk_retrieved_docs = retriever.get_relevant_documents(query_lst[1])
print(f"top-{k} similar document to the query '{query_lst[1]}':")
print(topk_retrieved_docs)

chromadb_client.delete_collection(name="tmp_collection")

top-5 similar document to the query 'पेरिस बहुत खूबसूरत शहर है.':
[Document(page_content='पेरिस बहुत खूबसूरत शहर है.'), Document(page_content='Paris is a very beautiful city.'), Document(page_content='Paris est une très belle ville.'), Document(page_content='París es una ciudad muy hermosa.'), Document(page_content='कोहेरे से नमस्ते!')]


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10, chunk_overlap=0, length_function=len)
rcts_docs = text_splitter.split_documents(text_doc)
print (f"You have {len(rcts_docs)} documents")
# print (rcts_docs)


openai_embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

chromadb_client = chromadb.Client()
db_openai_embed = Chroma.from_documents(
    rcts_docs, openai_embed_model, client=chromadb_client, collection_name="tmp_collection"
)

print("There are", db_openai_embed._collection.count(), "in the collection")

retriever = db_openai_embed.as_retriever(search_kwargs={"k": 5})
query= "How Google plans to challenge OpenAI?"
retrieved_docs = retriever.get_relevant_documents(query)
print(retrieved_docs[0].page_content)

chromadb_client.delete_collection(name="tmp_collection")

You have 124 documents
There are 124 in the collection
OpenAI


In [175]:
# from langchain.retrievers.web_research import WebResearchRetriever

# # Initialize components
# chat_openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
# search = GoogleSearchAPIWrapper()
# openai_embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# vectorstore = Chroma(embedding_function= openai_embed_model)

# # Instantiate WebResearchRetriever
# web_research_retriever = WebResearchRetriever.from_llm(vectorstore=vectorstore, llm=chat_openai, search=search)

# # Retrieve documents
# docs = web_research_retriever.get_relevant_documents("Who are competitors of ChatGPT?")
# print(docs)

# 3. Chains

The chains are responsible for creating an end-to-end pipeline for using the language models. They will join the model, prompt, memory, parsing output, and debugging capability and provide an easy-to-use interface. A chain will 
1. receive the user’s query as an input, 
2. process the LLM’s response, and lastly, 
3. return the output to the user.

## LLMChain

In [26]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()
template = """List five best words as substitute for {word} as comma separated."""
prompt=PromptTemplate(template=template, input_variables=['word'])

llm_chain = LLMChain(llm=llm_openai, prompt=prompt, output_parser=csv_parser)

llm_chain.predict(word= "Artificial")

['Intelligence\n\n1. Machine Learning\n2. Cognitive Computing\n3. Robotic Intelligence\n4. Automated Intelligence\n5. Smart Technology']

In [11]:
translation_template = """
Translate the following text from {source_language} to {target_language}: 
{text}:
translated_text:
"""
translation_prompt = PromptTemplate(template=translation_template, input_variables=["source_language", "target_language", "text"])
translation_chain = LLMChain(llm=llm_openai, prompt=translation_prompt)
source_language = "English"
target_language = "Spanish" # "Hindi"
text = "I don't know what your name is or where do you live or anything whatsoever."
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
translated_text

'No sé cuál es tu nombre o dónde vives o cualquier cosa en absoluto.'

In [13]:
template = """Question: {question}

Answer: """

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# create prompt template > LLM chain
llm_chain = LLMChain( prompt=prompt, llm=llm_openai)


qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
for res_one in res.generations:
    print(res_one[0].text)


The capital city of France is Paris.
The largest mammal on Earth is the blue whale.
 Nitrogen (N2) is the most abundant gas in Earth's atmosphere, making up about 78% of the total volume. Oxygen (O2) is the second most abundant gas, making up about 21% of the total volume. Other gases, such as argon, carbon dioxide, and water vapor, make up the remaining 1%.
A ripe banana is typically yellow in color.


In [8]:
# word_list= [{"word": "Artificial"}, {"word": "Behaviour"}, {"word": "Intelligent"}]
# response= llm_chain.generate(word_list)
# print(response)

generations=[[Generation(text='\n\nSynthetic, Man-made, Simulated, Faux, Imitation', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n1. Conduct\n2. Manner\n3. Demeanor\n4. Attitude\n5. Disposition', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n1. Clever, \n2. Resourceful, \n3. Astute, \n4. Ingenious, \n5. Discerning', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 36, 'completion_tokens': 71, 'total_tokens': 107}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('46319bc1-6cce-4e71-8fd4-e74f7f189b26')), RunInfo(run_id=UUID('21a3ec52-38f5-499f-8e6d-9a2d37ac04cb')), RunInfo(run_id=UUID('8a5aff9b-c4bd-4b73-b477-184fb9938ace'))]


## ConversationChain

In [227]:
conversation = ConversationChain(llm=llm_openai, memory=ConversationBufferMemory(), output_parser=csv_parser)

response= conversation.predict(input="List five best words as substitute for 'artificial' as comma separated.")
print(response)

response= conversation.predict(input="And the next 4 words.")
print(response)

['Synthetic', 'simulated', 'man-made', 'fabricated', 'ersatz.']
["'Imitation'", "'virtual'", "'constructed'", "'replicated.'"]


## Sequential Chain

In [234]:
# poet
poet_template = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template = PromptTemplate(template=poet_template, input_variables=["input"])

# creating the poet chain
poet_chain = LLMChain(llm=llm_openai, prompt=poet_prompt_template, output_key="poem")

# critic
critic_template = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template = PromptTemplate(template=critic_template, input_variables=["poem"])

# creating the critic chain
critic_chain= LLMChain(llm=llm_openai, prompt=critic_prompt_template, output_key="critic_verified")


overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

# Run the poet and critic chain with a specific theme
theme = "Love between two troubled souls"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)





poem: Love between two troubled souls,
A bond that's strong, yet full of holes.
They've both been hurt, they've both been scarred,
But somehow, together, they've found a shard.

They understand each other's pain,
And know that love can heal again.
They hold each other through the night,
And chase away each other's fright.

Their love is not a fairytale,
But it's real and raw, and it prevails.
They've been through storms, they've been through fire,
But their love only grows higher.

They may not have it all figured out,
But they know what love is all about.
It's not perfect, it's not easy,
But it's worth it, and it's oh so freeing.

Their troubled souls may never fully heal,
But together, they've found something real.
Love between two troubled souls,
A bond that's strong, and it only grows.

Romantic_expressions: True
Nature_descriptions: False


In [28]:
# Initialize the CSV parser
csv_parser = CommaSeparatedListOutputParser()

suggestions_template = """
You are an assistant that helps users to plan trips.
Suggest the best place to visit in {country} for someone who loves beach.
"""

suggestions_prompt = PromptTemplate(
    template=suggestions_template,
    input_variables=["country"]
)

# creating the suggestions chain
suggestions_chain = LLMChain(llm=llm_openai, prompt=suggestions_prompt, output_key="place")

itinerary_template = """You are an assistant that helps users to plan trips.
Give a brief itinerary for {place} of five days as a numbered list.
Itinerary:
"""

itinerary_prompt = PromptTemplate(template=itinerary_template, input_variables=["place"])

# creating the itinerary chain
itinerary_chain= LLMChain(llm=llm_openai, prompt=itinerary_prompt, output_key="detailed_itinerary")


overall_chain = SimpleSequentialChain(chains=[suggestions_chain, itinerary_chain])

country= "Mongolia"
itinerary = overall_chain.run(country)

# Print the review to see the critic's evaluation
print(itinerary)


1. Day 1: Arrival in Ulaanbaatar, the capital city of Mongolia. Explore the city's main attractions such as the Gandan Monastery, Sukhbaatar Square, and the National Museum of Mongolia.
2. Day 2: Take a scenic drive to Lake Khuvsgul, known as the "Blue Pearl of Mongolia". Enjoy a boat ride on the lake and take in the breathtaking views of the surrounding mountains.
3. Day 3: Visit Terkhiin Tsagaan Lake, also known as the "Great White Lake". Take a hike around the lake and enjoy a picnic lunch while admiring the stunning landscape.
4. Day 4: Head to the Orkhon River and spend the day fishing or rafting on its crystal clear waters. In the evening, experience a traditional Mongolian dinner and cultural performance.
5. Day 5: Return to Ulaanbaatar and spend the day shopping for souvenirs and trying out local cuisine before departing for your next destination.


## RetrievalQA Chain

This chain first does a retrieval step to fetch relevant documents, then passes those documents into an LLM to generate a response.

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, length_function=len)
rcts_docs = text_splitter.split_documents(text_doc)
print (f"You have {len(rcts_docs)} documents")
# print (rcts_docs)


openai_embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

chromadb_client = chromadb.Client()
db_openai_embed = Chroma.from_documents(
    rcts_docs, openai_embed_model, client=chromadb_client, collection_name="tmp_collection"
)

print("There are", db_openai_embed._collection.count(), "in the collection")

retriever = db_openai_embed.as_retriever()

# create a retrieval chain
qa_chain = RetrievalQA.from_chain_type(llm= llm_openai, chain_type="stuff", retriever=retriever)
query = "How Google plans to challenge OpenAI?"
response = qa_chain.run(query)
print(response)
chromadb_client.delete_collection(name="tmp_collection")


You have 12 documents
There are 12 in the collection

Google plans to challenge OpenAI by opening up its AI language model PaLM and launching an API for it, along with other AI enterprise tools.


In [1]:
qa_chain

NameError: name 'qa_chain' is not defined

### RetrievalQAWithSourcesChain

It does question answering over retrieved documents, and cites it sources. Use this when you want the answer response to have sources in the text response. 

There are many more chains such as, __TransformationChain, LLMCheckerChain, LLMSummarizationChain, Custom Chain, etc__.

# 4. Memory

In the ever-evolving world of chatbot applications, maintaining message history can be essential for delivering context-aware responses that enhance user experiences.

In LangChain, memory is a fundamental aspect of conversational interfaces, allowing systems to reference past interactions. This is achieved through storing and querying information, with two primary actions: reading and writing. The memory system interacts with a chain twice during a run, augmenting user inputs and storing the inputs and outputs for future reference.


* __Storing Chat Messages__: The LangChain memory module integrates various methods to store chat messages, ranging from in-memory lists to databases. This ensures that all chat interactions are recorded for future reference.
* __Querying Chat Messages__: Beyond storing chat messages, LangChain employs data structures and algorithms to create a useful view of these messages. Simple memory systems might return recent messages, while more advanced systems could summarize past interactions or focus on entities mentioned in the current interaction.

## ConversationBufferMemory

This memory implementation stores the entire conversation history as a single string. The advantages of this approach is maintains a complete record of  the conversation, as well as being straightforward to implement and use. On the other hands, It can be less efficient as the conversation grows longer and may lead to excessive repetition if the conversation history is too long for the model's token limit.

If the token limit of the model is surpassed, the buffer gets truncated to fit within the model's token limit. This means that older interactions may be removed from the buffer to accommodate newer ones, and as a result, the conversation context might lose some information.

To avoid surpassing the token limit, you can monitor the token count in the buffer and manage the conversation accordingly. For example, you can choose to shorten the input texts or remove less relevant parts of the conversation to keep the token count within the model's limit.

In [32]:
conversation = ConversationChain(llm=llm_openai, verbose=True)
# conversation = ConversationChain(llm=llm_openai, verbose=True, memory=ConversationBufferMemory())
output = conversation.predict(input="Hi there!")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?


In [29]:
output = conversation.predict(input="What is the largest retail company in the world?")
output = conversation.predict(input="Who is the CEO of the company and where is the headquarter?")
output = conversation.predict(input="Who are the competitors?")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: What is the largest retail company in the world?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by Op

The __ConversationChain__ uses the __ConversationBufferMemory__ class by default to provide a history of messages. This memory can save the previous conversations in form of variables. The class accepts the ___return_messages___ argument which is helpful for dealing with chat models. This is how the CoversationChain keep context under the hood.

In [30]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi there!"}, {"output": "Hi there! It's nice to meet you. How can I help you today?"})

print( memory.load_memory_variables({}) )

{'history': [HumanMessage(content='hi there!'), AIMessage(content="Hi there! It's nice to meet you. How can I help you today?")]}


Here we used __MessagesPlaceholder__ function to create a placeholder for the conversation history in a chat model prompt. It is particularly useful when working with __ConversationChain__ and __ConversationBufferMemory__ to maintain the context of a conversation. The __MessagesPlaceholder__ function takes a variable name as an argument, which is used to store the conversation history in the memory buffer.


In [42]:
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm_openai, memory=memory, prompt=prompt, verbose=True)

user_message = "What was the most important research paper published in the field of Natural Language Processing?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?

> Finished chain.
{'input': 'What was the most important research paper published in the field of Natural Language Processing?', 'history': [HumanMessage(content='What was the most important research paper published in the field of Natural Language Processing?'), AIMessage(content=' \n\nAI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.')], 'response': ' \n\nAI: According to my database, the most important research paper published in the field of

In [43]:
user_message = "Please give a brief description of the various building blocks of that paper."
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?
AI:  

AI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.
Human: Please give a brief description of the various building blocks of that paper.

> Finished chain.
{'input': 'Please give a brief description of the various building blocks of that paper.', 'history': [HumanMessage(content='What was the most important research paper published in the field of Natural Language Processing?'), AIMessage(content=' \n\nAI: According to my database, the most 

In [44]:
user_message = "When was this paper published?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?
AI:  

AI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.
Human: Please give a brief description of the various building blocks of that paper.
AI: 
AI: The Transformer model introduced in the paper consists of two main building blocks: the self-attention mechanism and the feed-forward neural network. The self-attention mechanism allows the model to focus on different parts of the input sequence, while the feed-forward neural network helps to captu

In a scenario where a conversation has a large sum of tokens, the computational cost and resources required for processing the conversation will be higher. This highlights the importance of managing tokens effectively. Strategies for achieving this include limiting memory size through methods like __ConversationBufferWindowMemory__ or summarizing older interactions using __ConversationSummaryBufferMemory__. These approaches help control the token count while minimizing associated costs and computational demands in a more efficient manner.

## ConversationBufferWindowMemory

This class limits memory size by keeping a list of the most __recent K__ interactions. It maintains a sliding window of these recent interactions, ensuring that the buffer does not grow too large. 

Basically, this implementation stores a fixed number of recent messages in the conversation that makes it more efficient than __ConversationBufferMemory__.  

Also, it reduces the risk of exceeding the model's token limit. However, the downside of using this approach is that it does not maintain the complete conversation history. 

The chatbot might lose context if essential information falls outside the fixed window of messages.

It is possible to retrieve specific interactions from ConversationBufferWindowMemory. 

In [47]:
from langchain import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferWindowMemory(k=2, return_messages=True)
conversation = ConversationChain(llm=llm_openai, memory=memory, prompt=prompt, verbose=True)

user_message = "What was the most important research paper published in the field of Natural Language Processing?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?

> Finished chain.
{'input': 'What was the most important research paper published in the field of Natural Language Processing?', 'history': [], 'response': ' \n\nAI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.'}


In [48]:
user_message = "Please give a brief description of the various building blocks of that paper."
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?
AI:  

AI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.
Human: Please give a brief description of the various building blocks of that paper.

> Finished chain.
{'input': 'Please give a brief description of the various building blocks of that paper.', 'history': [HumanMessage(content='What was the most important research paper published in the field of Natural Language Processing?'), AIMessage(content=' \n\nAI: According to my database, the most 

In [49]:
user_message = "When was this paper published?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: What was the most important research paper published in the field of Natural Language Processing?
AI:  

AI: According to my database, the most important research paper published in the field of Natural Language Processing is "Attention is All You Need" by Vaswani et al. This paper introduced the Transformer model, which has greatly advanced the field of NLP and has been used in various applications such as machine translation and text summarization.
Human: Please give a brief description of the various building blocks of that paper.
AI: 
AI: The Transformer model introduced in the paper consists of two main building blocks: the self-attention mechanism and the feed-forward neural network. The self-attention mechanism allows the model to focus on different parts of the input sequence, while the feed-forward neural network helps to captu

In [50]:
user_message = "Who were the authors of this paper?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Please give a brief description of the various building blocks of that paper.
AI: 
AI: The Transformer model introduced in the paper consists of two main building blocks: the self-attention mechanism and the feed-forward neural network. The self-attention mechanism allows the model to focus on different parts of the input sequence, while the feed-forward neural network helps to capture complex relationships between words. Additionally, the paper also introduced the concept of positional encoding, which helps the model to understand the order of words in a sentence. These building blocks have been widely adopted in subsequent NLP research and have greatly improved the performance of language models.
Human: When was this paper published?
AI:  
AI: The paper was published in 2017 at the Conference on Neural Information Processing Systems (

In [52]:
user_message = "What was the first question I asked?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Who were the authors of this paper?
AI: 
AI: The authors of this paper were Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.
Human: What was the name of the paper you had mentioned?
AI: 
AI: The name of the paper is "Attention is All You Need".
Human: What was the first question I asked?

> Finished chain.
{'input': 'What was the first question I asked?', 'history': [HumanMessage(content='Who were the authors of this paper?'), AIMessage(content='\nAI: The authors of this paper were Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.'), HumanMessage(content='What was the name of the paper you had mentioned?'), AIMessage(content='\nAI: The name of the paper is "Attention is All You Need".')

## ConversationSummaryMemory

__ConversationSummaryMemory__ is a memory management strategy that reates a summary of the conversation over time, useful for condensing information from longer conversations. 

It extracts key information from previous interactions and condenses it into a shorter, more manageable format.  

Here is a list of pros and cons of ConversationSummaryMemory.

* __Pros__:

    * __Condensing conversation information__: By summarizing the conversation, it helps reduce the number of tokens required to store the conversation history, which can be beneficial when working with token-limited models like GPT-3
    * __Flexibility__: You can configure this type of memory to return the history as a list of messages or as a plain text summary. This makes it suitable for chatbots.
    * __Direct summary prediction__: The predict_new_summary method allows you to directly obtain a summary prediction based on the list of messages and the previous summary. This enables you to have more control over the summarization process.


* __Cons__:

    * __Loss of information__: Summarizing the conversation might lead to a loss of information, especially if the summary is too short or omits important details from the conversation.
    * __Increased complexity__: Compared to simpler memory types like ConversationBufferMemory, which just stores the raw conversation history, ConversationSummaryMemoryrequires more processing to generate the summary, potentially affecting the performance of the chatbot. 


## ConversationSummaryBufferMemory

__ConversationSummaryBufferMemory__ is a memory management strategy that combines the ideas of keeping a buffer of recent interactions in memory and compiling old interactions into a summary.

In [54]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

memory=ConversationSummaryBufferMemory(llm=llm_openai, max_token_limit=40)
conversation_with_summary = ConversationChain(llm=llm_openai, memory=memory, verbose=True)
output = conversation_with_summary.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?


In [56]:
output = conversation_with_summary.predict(input="What is the largest retail company in the world?")
output = conversation_with_summary.predict(input="Who is the CEO of the company and where is the headquarter?")
output = conversation_with_summary.predict(input="Who are the competitors?")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI responds by introducing itself as an AI created by OpenAI. The AI explains that it is constantly learning and improving through machine learning algorithms and asks how it can assist the human.
Human: What is the largest retail company in the world?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and 

## VectorStoreRetrieverMemory

VectorStoreRetrieverMemory stores memories in a vector store and queries the top-K most "salient" documents every time it is called. This memory type doesn't explicitly track the order of interactions but uses vector retrieval to fetch relevant memories.



In [58]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

# Initialize your vector store (specifics depend on the chosen vector store)
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

embedding_size = 1536  # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

# Create your VectorStoreRetrieverMemory
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# Save context and relevant information to the memory
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't like the Celtics"}, {"output": "ok"})

# Retrieve relevant information from memory based on a query
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

# 5. Tools

Tools are instrumental in connecting the language model with other sources of data or computation, including search engines, APIs, and other data repositories. Language models can only access the knowledge they've been trained on, which can quickly become obsolete.

Tools are modular, reusable components meticulously designed to accomplish specific tasks or provide answers to distinct types of questions. By integrating these tools seamlessly into the system, users can effortlessly tap into a diverse range of functionalities and information sources to tackle challenges and generate meaningful responses.


A few notable examples of tools in LangChain are:
* __Google Search__: This tool uses the Google Search API to fetch relevant information from the web, which can be used to answer queries related to current events, facts, or any topic where a quick search can provide accurate results.
* __Requests__: This tool employs the popular Python library "requests" to interact with web services, access APIs, or obtain data from different online sources. It can be particularly useful for gathering structured data or specific information from a web service.
* __SerpAPI__: This tool is used to scrape results from Google and other search engines. It is the JSON representative of Google search result and other search engine.

* __Python REPL__: The Python REPL (Read-Eval-Print Loop) tool allows users to execute Python code on-the-fly to perform calculations, manipulate data, or test algorithms. It serves as an interactive programming environment within the LangChain system.
* __Wikipedia__: The Wikipedia tool leverages the Wikipedia API to search and retrieve relevant articles, summaries, or specific information from the vast repository of knowledge on the Wikipedia platform.
* __Wolfram Alpha__: With this tool, users can tap into the powerful computational knowledge engine of Wolfram Alpha to answer complex questions, perform advanced calculations, or generate visual representations of data.


### GoogleSearchAPIWrapper

In [88]:
search= GoogleSearchAPIWrapper()
search.results("Who is the CEO of Walmart?", 3)
# search.results("What is Langchain?", 2)

[{'title': 'Introduction | 🦜️ Langchain',
  'link': 'https://python.langchain.com/docs/get_started/introduction',
  'snippet': 'LangChain is a framework for developing applications powered by language models. It enables applications that:'},
 {'title': 'What Is LangChain and How to Use It: A Guide',
  'link': 'https://www.techtarget.com/searchenterpriseai/definition/LangChain',
  'snippet': 'Why is LangChain important? LangChain is a framework that simplifies the process of creating generative AI application interfaces. Developers working on these\xa0...'}]

### SerpAPIWrapper

In [91]:
from langchain_community.utilities import SerpAPIWrapper
search = SerpAPIWrapper()
print(search.run("Who is the CEO of Walmart?"))
print(search.run("What is Langchain?"))


Doug McMillon
["LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.", 'LangChain entity_type: kp3_verticals.', 'LangChain kgmid: /g/11kjpl7_60.', 'LangChain initial_release_date: October 2022.', 'LangChain developer_s: Harrison Chase.', 'LangChain license: MIT License.', 'LangChain repository: github.com/langchain-ai/langchain.', 'LangChain stable_release: 0.1.8 / 19 February 2024; 24 days ago.', 'LangChain written_in: Python and JavaScript.', 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'LangChain provides AI developers with tools to connect language models with external data sources. It is open-source and supported by an active community.', "LangChain uses the power of 

### PythonREPL

In [78]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_code= """
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

print(fib(5))
print(fib(10))
"""
print(python_repl.run("print(1+1)"))
print(python_repl.run(python_code))


2

5
55



### WolframAlphaAPIWrapper

In [65]:
from langchain. utilities.wolfram_alpha import WolframAlphaAPIWrapper

wolfram = WolframAlphaAPIWrapper()
result = wolfram.run("What is 2x+5 = -3x + 7?")
print(result)  # Output: 'x = 2/5'

Assumption: 2 x + 5 = -3 x + 7 
Answer: x = 2/5


### WikipediaAPIWrapper

In [67]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("Elon Musk")


"Page: Elon Musk\nSummary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylva

### Shell (bash)

In [72]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

result = shell_tool.run({"commands": ["echo 'Hello World!'", "ls"]})
print(result)

Executing command:
 ["echo 'Hello World!'", 'ls']
Hello World!
chroma
langchain_fundamentals.ipynb
llama.cpp
llm_101.ipynb
models



Complete list of tools can be found here: https://python.langchain.com/docs/integrations/tools/

# 6. Agents

We can abstract two primary modes of operation to consider when employing an LLM: as a __content generator__ and as a __reasoning engine__.

* When used as a __content generator__, the language model is asked to create content entirely from its internal knowledge base. This approach can lead to highly creative outputs but can also result in unverified information or 'hallucinations' due to the model's reliance on pre-trained knowledge.
* On the other hand, when functioning as a __reasoning engine__, the Agent acts more as an information manager rather than a creator. In this mode, it is tasked with gathering relevant, accurate information, often aided by external tools. This involves the LLM drawing from similar resources on a given topic and constructing new content by extracting and summarizing the relevant details.


LangChain introduces a powerful concept called __Agents__ that takes the idea of chains to a whole new level. Agents leverage language models to dynamically determine sequences of actions to perform, making them incredibly versatile and adaptive. Unlike traditional chains, where actions are hardcoded in code, agents employ language models as reasoning engines to decide which actions to take and in what order.

An agent has access to a suite of tools and can decide which of these tools to call, depending on the user input. Tools are functions that perform specific duties. To create an agent in LangChain, you can use the __initialize_agent__ function along with the __load_tools__ function to prepare the tools the agent can use. 

The Agent is the core component responsible for decision-making. It harnesses the power of a language model and a prompt to determine the next steps to achieve a specific objective. The inputs to an agent typically include:

* __Tools__: Descriptions of available tools .
* __User Input__: The high-level objective or query from the user.
* __Intermediate Steps__: A history of (action, tool output) pairs executed to reach the current user input.

In the context of language models, agents are used to decide the course of action and the sequence of these actions. These actions can either be the utilization of a tool, observing its output, or offering a response to the user. The real potential of agents unfolds when they are utilized appropriately. This explanation aims to simplify the usage of agents via the highest-level API.

Presently, most of the agents in LangChain fall into one of these two categories:

__Action Agents__: These agents determine and execute a single action. They are typically used for straightforward tasks.
__Plan-and-Execute Agents__: These agents first devise a plan comprising multiple actions and then execute each action sequentially. They are more suited for complex or long-running tasks as they help maintain focus on long-term objectives.
While Action Agents are more traditional and suitable for smaller tasks, Plan-and-Execute Agents help maintain long-term objectives and focus. However, they might lead to more calls and higher latency. Often, it's beneficial to let an Action Agent manage the execution for the Plan-and-Execute agent, thus utilizing both strengths.

For example, a high-level workflow of Action Agents would look something like this:

* The agent receives user input.
* It decides which tool to use (if any) and determines its input.
* The chosen tool is called with the provided input, and an observation (the output of the tool) is recorded.
* The history of the tool, tool input, and observation are relayed back to the agent, which then decides the next step.
* This process is repeated until the agent no longer needs to use a tool, at which point it directly responds to the user.

In [104]:
# Importing necessary modules
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# Loading some tools to use. The llm-math tool uses an LLM, so we pass that in.
tools = load_tools(["serpapi", "wolfram-alpha"])

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm_openai, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Testing the agent
query = "What's the result of 1000 plus the total number of goals scored in the men's football world cup in 2022?"
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
 I should use wolfram_alpha to get the answer.
Action: wolfram_alpha
Action Input: 1000 + total number of goals scored in men's football world cup 2022
Observation: Wolfram Alpha wasn't able to answer it
Thought: I should try using Search instead.
Action: Search
Action Input: "1000 + total number of goals scored in men's football world cup 2022"
Observation: A total of 172 goals were scored during the 2022 World Cup in Qatar, marking a new record for the tournament. This was three more goals than the previous tournament in 2018.
Thought: I now know the final answer.
Final Answer: The result of 1000 plus the total number of goals scored in the men's football world cup in 2022 is 1172.

> Finished chain.
The result of 1000 plus the total number of goals scored in the men's football world cup in 2022 is 1172.


Let's break down the steps to see how the agent functions as a "reasoning engine":

* __Query Processing__: The agent receives a query: "What's the result of 1000 plus the total number of goals scored in the men's football world cup in 2022?” The agent identifies two distinct tasks within this query - finding out the number of goals scored in the 2018 soccer world cup and adding 1000 to such number.
* __Tool Utilization__: The agent uses the "serpapi" or "google-search" tool to answer the first part of the query. This is an example of the agent using external tools to gather accurate and relevant information. The agent isn't creating this information; it's pulling the data from an external source.
* __Information Processing__: For the second part of the query, the agent uses the "wolfram-alpha" tool to perform a sum reliably. Again, the agent isn't creating new information. Instead, it's processing the data it has gathered.
* __Synthesis and Response__: After gathering and processing the information, the agent synthesizes it into a coherent response that answers the original query.

In this way, the agent acts as a __"reasoning engine"__. It's not generating content from scratch but rather gathering, processing, and synthesizing existing information to generate a response.  This approach allows the agent to provide accurate and relevant responses, making it a powerful tool for tasks that involve data retrieval and processing.


### Python-REPL

In [1]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
tools= [repl_tool]
agent = initialize_agent(tools, llm_openai, agent="chat-zero-shot-react-description", verbose=True, handle_parsing_errors=True)
query= "Write a code to generate a list of all prime numbers between 1 to 100 and then sort it in descedning order and then print the list."
response=  agent.run(query) 
print(response)

NameError: name 'initialize_agent' is not defined

In [130]:
prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text in a bulleted list format in not more than five sentences: {query}"
)

summarize_chain = LLMChain(llm=llm_openai, prompt=prompt)

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

agent = initialize_agent(tools, llm_openai, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

query= "Write history of Walmart and summarize in a bulleted list format in not more than five sentences."
response=  agent.run(query) 
print(response)
# response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
# print(response['output'])




> Entering new AgentExecutor chain...
 I should use the Search tool to find information about Walmart's history.
Action: Search
Action Input: "History of Walmart"
Observation: the 1960s. Retail Revolution. Sam Walton's strategy is built on an unshakeable foundation: the lowest prices anytime, anywhere. 1962. On July 2, 1962, Sam ... On July 2, 1962, Sam Walton opened the first Walmart store in Rogers, AR. By 1969, the company was officially incorporated and registering $12.7 billion in ... Jan 26, 2024 ... Walmart is an American operator of discount stores; it is one of the world's biggest retailers and among the world's largest corporations. Jan 2, 2020 ... Walmart doubled its sales in 1995 three years after the death of founder Sam Walton. Company management had taken a risk in doing so, electing ... The Walmart chain proper was founded in 1962 with a single store in Rogers, expanding inside Oklahoma by 1968 and throughout the rest of the Southern United ... Store purchases can be 

### Pandas DataFrame Agent

### SQL Database Agent

